# `smlb` mini demonstration:<br>The watercolor pigments 2019 dataset

Scientific Machine Learning Benchmark:<br>
A benchmark of regression models in chem- and materials informatics.<br>
2019-2020, Citrine Informatics.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import smlb

Load the dataset, which comes with a featurizer. Use tab completion to find the right import:

In [ ]:
from smlb.datasets.experimental.watercolor_pigments_c19.watercolor_pigments_c19 import \
    WatercolorPigments2019Dataset, WatercolorPigments2019DatasetFeatures

We use standard learners from `scikit-learn` and a simple learning curve workflow:

In [ ]:
from smlb.learners.scikit_learn.gaussian_process_regression_sklearn import GaussianProcessRegressionSklearn
from smlb.learners.scikit_learn.random_forest_regression_sklearn import RandomForestRegressionSklearn
from smlb.learners.scikit_learn.extremely_randomized_trees_regression_sklearn import ExtremelyRandomizedTreesRegressionSklearn

from smlb.workflows.learning_curve_regression import LearningCurveRegression

## The data

Get information about the dataset.
Note that references are given.

In [ ]:
print(WatercolorPigments2019Dataset.__doc__) # using `print` instead of `help` avoids clutter

There are 13 primary pigments:

In [ ]:
color_names = WatercolorPigments2019Dataset.COLOR_NAMES[1:]  # indexing starts with 1, as in the original reference
print(pd.DataFrame(color_names, index=range(1,13+1), columns=['name']))

Since watercolors are semi-transparent, the color of these pigments depends on their amount:

In [ ]:
# build a table mapping index and concentration to color
dataset = WatercolorPigments2019Dataset(filter_="primary")  # only primary pigments
color_table = dict()
for entry, rgb in zip(dataset.samples(), dataset.labels()):
    color_table[entry['index'], entry['concentration']] = rgb

In [ ]:
# visualize
(fig,ax) = plt.subplots(figsize=(14,8))
concentrations = np.unique(tuple(key[1] for key in color_table.keys()))
rect_xsize, rect_ysize, pad_x, pad_y = 10, 10, 2, 2
ax.set_xlim(-rect_xsize, len(concentrations)*(rect_xsize+pad_x)+pad_x)
ax.set_ylim(rect_ysize, -13*(rect_ysize+pad_y)-pad_y)
ax.text(-rect_xsize/2, rect_ysize/2, 'Color', ha='center', weight='bold')
for j, c in enumerate(concentrations):
    ax.text(j*(rect_xsize+pad_x)+rect_xsize/2, rect_ysize/2, f'{c} mL', ha='center', weight='bold')
for i in range(1, 13+1):
    ax.text(-rect_xsize/2, -i*(rect_ysize+pad_y)+rect_ysize/2, str(i), weight='bold')
    for j, c in enumerate(concentrations):
        ax.add_patch(mpl.patches.Rectangle((j*(rect_xsize+pad_x), -i*(rect_ysize+pad_y)), rect_xsize, rect_ysize, color=color_table[i,c] / 255, fill=True))
ax.invert_yaxis()
ax.axis('off')
plt.show()

The mapping from concentration to color channel is non-linear:

Note that below plot uses a single color for each line; while it is
possible to draw line gradients via `matplotlib.collections.LineCollection`,
the visual effect is rather limited and hard to see.

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,12))
concentrations = (0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.12, 0.16)

for index in range(1, 13+1):
    colors = np.asarray([color_table[index, c] for c in concentrations])
    ax[0].plot(concentrations, colors[:,0], color=colors[0]/255)
    ax[1].plot(concentrations, colors[:,1], color=colors[0]/255)
    ax[2].plot(concentrations, colors[:,2], color=colors[0]/255, label=index)

ax[0].set_xlabel('concentration / mL'); ax[0].set_ylabel('red')
ax[1].set_xlabel('concentration / mL'); ax[1].set_ylabel('green')
ax[2].set_xlabel('concentration / mL'); ax[2].set_ylabel('blue')

ax[2].legend(loc=(1.05, 0.05))
plt.show()

## A: only mixtures, one-hot encoding, random sampling

The default initializer parameters of the water colors dataset yield only mixtures (no primary pigments) via a one-hot encoding.

In [ ]:
dataset = WatercolorPigments2019Dataset(labelf=lambda arg: arg[0])
features = WatercolorPigments2019DatasetFeatures()

We use 15% of data for validation.
Training set sizes are equi-distant in log-space.

In [ ]:
n, m = dataset.num_samples, round(dataset.num_samples * 0.15)
validation = smlb.RandomSubsetSampler(size=m, rng=0)

training_sizes = np.asarray( np.logspace(1, np.log10(n-m), num=6), dtype=int )
print(training_sizes)
training = tuple(smlb.RandomSubsetSampler(size=n, rng=0) for n in training_sizes)

In [ ]:
import sklearn as skl
kernel = skl.gaussian_process.kernels.DotProduct() + skl.gaussian_process.kernels.WhiteKernel()
kernel = skl.gaussian_process.kernels.RBF(length_scale=1, length_scale_bounds='fixed') + skl.gaussian_process.kernels.WhiteKernel()

In [ ]:
learner_gpr_skl = GaussianProcessRegressionSklearn(kernel=kernel, random_state=0) # default is Gaussian kernel
learner_rf_skl = RandomForestRegressionSklearn(random_state=0)
learner_ert_skl = ExtremelyRandomizedTreesRegressionSklearn(random_state=0, uncertainties='naive')

In [ ]:
fig, ax = plt.subplots()
learning_curve = smlb.LearningCurvePlot(target=ax, axes_labels=("training set size", "root mean-squared error"))
workflow = LearningCurveRegression(data=dataset, features=features, training=training, validation=validation, 
                                   learners=[learner_rf_skl, learner_ert_skl, learner_gpr_skl], evaluations=[learning_curve])
workflow.run()
ax.legend(["Random forest", "Extremely randomized trees", "Gaussian process"], loc=(1.05, 0.05));

The Gaussian process fails with constant predictions, resulting in a flat learning curve at high error.

This seems to be a problem of hyperparameter optimization: When trained with a Gaussian kernel and 
additive Gaussian noise with fixed hyperparameters (length scale 0.05, noise level 0.1), performance
is much better, albeit still worse than random forests.